# Лабораторная работа No 6. RSA.
    выполнил Сергей Харитонов ПВ-32 (Вариант 14)

# Цель работы
    освоить алгоритм RSA

In [1]:
# Сначала реализуем классы ключей
class PrivatKey:
    
    def __init__(self, d: int, n: int):
        self.d = d
        self.n = n
    
    def __str__(self):
        return f"Приватный ключ ({self.d}; {self.n})"
    
    def decode(self, x: int):
#         возводим x в степень d по модулю n для декодирования 
        return pow(x, self.d, self.n)

class PublicKey:
    
    def __init__(self, e: int, n: int):
        self.e = e
        self.n = n

    def __str__(self):
        return f"Публичный ключ ({self.e}; {self.n})"
        
    def code(self, x: int):
#         возводим x в степень e по модулю n для декодирования 
        return pow(x, self.e, self.n)

In [2]:
# теперь реализуем все необходимые функции для получения пары ключей

# получение простых чисел в заданном диапазоне (решето эратосфена)
def get_prime_numbers(start: int, stop: int) -> list:
    a = [i for i in range(stop+1)]
    a[1] = 0
    primes = []

    i = 2
    while i <= stop:
        if a[i] != 0:
            primes.append(a[i])
            for j in range(i, stop+1, i):
                a[j] = 0
        i += 1
    
    while primes and primes[0] < start:
        primes.pop(0)
    
    return primes

# возвращает True если числа взаимно простые
def check_relative_prime(a: int, b: int):
    for divisor in range(2, min(a, b) + 1):
        if a % divisor == b % divisor == 0:
            return False
    return True

In [3]:
from random import choice
# создает приватный и публичный ключ с n заданного размера
def get_rsa_key(length) -> (PublicKey, PrivatKey):
    
#     находим границы для n через битовые сдвиги
    n_min = 1 << (length * 2 - 1)
    n_max = (1 << length * 2) - 1

#   находим границы для p и q
    start = 1 << (length - 1)
    stop = 1 << (length + 1)
    prime_numbers = get_prime_numbers(start, stop)

#     случайно выбираем из p и q подходящую пару
    while prime_numbers:
        p = choice(prime_numbers)
        prime_numbers.remove(p)
        q_s = [q for q in prime_numbers
                        if n_min <= p * q <= n_max]
        if q_s:
            q = choice(q_s)
            break
    
#     Вычисляем фи(n) что бы найти e которое меньше его
    f_n = (p - 1) * (q - 1)
    for e in range(3, f_n):
#         e и фи(n) должны быть взаимно простыми
        if check_relative_prime(e, f_n):
            break

#     находим d удовлетворяющее условию d * e по модулю фи(n) = 1
    for d in range(3, f_n):
        if d * e % f_n == 1:
            break

#     генерируем два ключа Публичный и Приватный
    return PublicKey(e, p*q), PrivatKey(d, p*q)

In [4]:
message = "error"
key_size = 4
public, privat = get_rsa_key(key_size)

int_message = [ord(sym) for sym in message]
code_int_message = [public.code(el) for el in int_message]
decode_int_message = [privat.decode(el) for el in code_int_message]
decode_message = ''.join([chr(el) for el in decode_int_message])

print(f"Сообщение: {message}")
print()
print(f"Размер ключа: {key_size}")
print(f"Сгенерированный приватный ключ: {str(privat)}")
print(f"Сгенерированный публичный ключ: {str(public)}")
print()
print(f"Сообщение в числах: {int_message}")
print(f"Закодированное сообщение в числах: {code_int_message}")
print()
print(f"Декодированное сообщение в числах: {decode_int_message}")
print(f"Декодированное сообщение: {decode_message}")

Сообщение: error

Размер ключа: 4
Сгенерированный приватный ключ: Приватный ключ (147; 253)
Сгенерированный публичный ключ: Публичный ключ (3; 253)

Сообщение в числах: [101, 114, 114, 111, 114]
Закодированное сообщение в числах: [85, 229, 229, 166, 229]

Декодированное сообщение в числах: [101, 114, 114, 111, 114]
Декодированное сообщение: error
